<td>
   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a>
</td>


<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/develop/examples/integrations/yolo/import_yolo_annotations.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>
<a href="https://github.com/Labelbox/labelbox-python/tree/develop/examples/integrations/yolo/import_yolo_annotations.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

# Import YOLOv8 annotations
This notebook will provides examples of setting up a Project with annotations generated with YOLOv8. We will be using the [Ultralytics](https://docs.ultralytics.com/) library to generate our annotations. In this guide we will be:
* Importing a demo image data rows that will be labeled
* Setting up our ontology that matches our YOLOv8 annotations
* Importing our data rows and attaching our ontology to a project
* Running our images through Ultralytics
* Importing the annotations generated


## Set up

In [ ]:
%pip install -q --upgrade "labelbox[data]"
%pip install -q --upgrade ultralytics

In [ ]:
import labelbox as lb
import labelbox.types as lb_types

import ultralytics
from PIL import Image

import uuid
import io

## API key and client
Provide a valid API key below in order to properly connect to the Labelbox client. Please review [Create API key guide](https://docs.labelbox.com/reference/create-api-key) for more information.

In [ ]:
API_KEY = None
client = lb.Client(api_key=API_KEY)

## Set up a YOLOv8 model
Below we will be initializing our model to be used for on our image data rows. We are using `yolov8n-seg.pt` since it supports segmentation masks. 

In [ ]:
model = ultralytics.YOLO("yolov8n-seg.pt")

## Example: Import YOLOv8 Annotations

The first few steps of this guide will demonstrating a basic workflow of creating data rows and setting up a project. For a quick complete overview of this process visit our [Quick start](https://docs.labelbox.com/reference/quick-start) guide.

### Importing an image data row

We will be using this [image](https://storage.googleapis.com/labelbox-datasets/image_sample_data/2560px-Kitano_Street_Kobe01s5s4110.jpeg) to be annotated with YOLOv8. Which has a lot of objects that can be picked up by YOLOv8. Later in this guide we will go into more detail on the exact annotations.

In [ ]:
global_key = str(uuid.uuid4())

# create data row
data_row = {
    "row_data":
        "https://storage.googleapis.com/labelbox-datasets/image_sample_data/2560px-Kitano_Street_Kobe01s5s4110.jpeg",
    "global_key":
        global_key,
    "media_type":
        "IMAGE",
}

# create dataset and import data row
dataset = client.create_dataset(name="YOLOv8 Demo Dataset")
task = dataset.create_data_rows([data_row])
task.wait_till_done()

print(f"Errors: {task.errors}")

### Setting up an ontology and a project
You must create a matching ontology and project with the data rows you are trying to label. The ontology should include the annotations that your are wanting to derive from YOLOv8. We will be introduce and explain a class mapping later in this guide so feel free to name your ontology features anything you want. In our example, we will be including a combination of bounding boxes, segment mask, and polygon tools to demonstrate converting each of those type of annotations from YOLOv8. Labelbox does not support ontologies were the same feature name is present at the first level so each of our feature names need to be unique.


#### Create an ontology

In [ ]:
ontology_builder = lb.OntologyBuilder(tools=[
    lb.Tool(tool=lb.Tool.Type.BBOX, name="Vehicle_bbox"),
    lb.Tool(tool=lb.Tool.Type.BBOX, name="Person_bbox"),
    lb.Tool(tool=lb.Tool.Type.RASTER_SEGMENTATION, name="Vehicle_mask"),
    lb.Tool(tool=lb.Tool.Type.RASTER_SEGMENTATION, name="Person_mask"),
    lb.Tool(tool=lb.Tool.Type.POLYGON, name="Vehicle_polygon"),
    lb.Tool(tool=lb.Tool.Type.POLYGON, name="Person_polygon"),
])

ontology = client.create_ontology(
    name="YOLOv8 Demo Ontology",
    normalized=ontology_builder.asdict(),
    media_type=lb.MediaType.Image,
)

#### Create and set up a project

In [ ]:
project = client.create_project(name="YOLOv8 Demo Project",
                                media_type=lb.MediaType.Image)

project.create_batch(name="batch 1", global_keys=[global_key])

project.setup_editor(ontology)

### Export our data rows and getting our predictions
In the step below, we are exporting our data row from our project and then adding the `row_data` and `global_key` to a list to then be used to make our predictions.

#### Export data

In [ ]:
export_task = project.export()
export_task.wait_till_done()

# prediction list we will be populating
url_list = []
global_keys = []


# callback that is ran on each data row
def export_callback(output: lb.BufferedJsonConverterOutput):

    data_row = output.json

    url_list.append(data_row["data_row"]["row_data"])

    global_keys.append(data_row["data_row"]["global_key"])


# check if export has errors
if export_task.has_errors():
    export_task.get_buffered_stream(stream_type=lb.StreamType.ERRORS).start()

if export_task.has_result():
    export_task.get_buffered_stream().start(stream_handler=export_callback)

### Import YOLOv8 annotations to a project
Now that you have finished your initial set up we create our predictions from YOLOv8 and import our annotations towards are project. We will be doing the following in this step:
1. Defining our import functions
2. Creating our labels
3. Importing our labels as either ground truths or MAL labels (pre-labels)

#### Defining our import functions
YOLOv8 supports a wide range of annotations. This guide shows importing bounding boxes, polygons and segment masks which matches our ontology. Below our the functions used for each type. These functions follow the same similar style, essentially, navigating through our result payload from YOLOv8 and converting it to the Labelbox annotation format. All of our functions support a class mapping which maps YOLOv8 annotation names to Labelbox feature names. The reason we have this mapping is to support having different names for Labelbox features compared to YOLOv8 annotation names. It also allows us to map common YOLOv8 names to the same Labelbox feature attached to our ontology. We will define this mapping first. In our case, we are mapping `bus` and `truck` to our Labelbox feature name `Vehicle` and `person` to our Labelbox feature name `Person`. We will create a mapping per tool type.

In [ ]:
bbox_class_mapping = {"person": "Person_bbox", "bus": "Vehicle_bbox", "truck": "Vehicle_bbox"}
mask_class_mapping = {"person": "Person_mask", "bus": "Vehicle_mask", "truck": "Vehicle_mask"}
polygon_class_mapping = {"person": "Person_polygon", "bus": "Vehicle_polygon", "truck": "Vehicle_polygon"}

##### Bounding box

In [ ]:
def get_yolo_bbox_annotation_predictions(
        yolo_results, model,
        ontology_mapping: dict[str:str]) -> list[lb_types.ObjectAnnotation]:
    """Convert YOLOV8 model bbox prediction results to labelbox annotations format

    Args:
        yolo_results (Results): YOLOv8 prediction results.
        model (Model): YOLOv8 model.
        ontology_mapping (dict[<yolo_class_name>: <labelbox_feature_name>]): Allows mapping between YOLOv8 class names and different Labelbox feature names.
    Returns:
        list[lb_types.ObjectAnnotation]
    """
    annotations = []

    for yolo_result in yolo_results:
      for bbox in yolo_result.boxes:
          class_name = model.names[int(bbox.cls)]

          # ignore bboxes that are not included in our mapping
          if not class_name in ontology_mapping.keys():
              continue

          # get bbox coordinates
          start_x, start_y, end_x, end_y = bbox.xyxy.tolist()[0]

          bbox_source = lb_types.ObjectAnnotation(
              name=ontology_mapping[class_name],
              value=lb_types.Rectangle(
                  start=lb_types.Point(x=start_x, y=start_y),
                  end=lb_types.Point(x=end_x, y=end_y),
              ),
          )

          annotations.append(bbox_source)

    return annotations

##### Segment Mask

In [ ]:
def get_yolo_segment_annotation_predictions(
        yolo_results, model,
        ontology_mapping: dict[str:str]) -> list[lb_types.Label]:
    """Convert YOLOV8 segment mask prediction results to labelbox annotations format

    Args:
        yolo_results (Results): YOLOv8 prediction results.
        model (Model): YOLOv8 model.
        ontology_mapping (dict[<yolo_class_name>: <labelbox_feature_name>]): Allows mapping between YOLOv8 class names and different Labelbox feature names.
    Returns:
        list[lb_types.ObjectAnnotation]
    """
    annotations = []

    for yolo_result in yolo_results:
      for i, mask in enumerate(yolo_result.masks.data):
          class_name = model.names[int(yolo_result.boxes[i].cls)]

          # ignore segment masks that are not included in our mapping
          if not class_name in ontology_mapping.keys():
              continue

          # get binary numpy array to byte array. You must resize mask to match image.
          mask = (mask.numpy() * 255).astype("uint8")
          img = Image.fromarray(mask, "L")
          img = img.resize((yolo_result.orig_shape[1], yolo_result.orig_shape[0]))
          img_byte_arr = io.BytesIO()
          img.save(img_byte_arr, format="PNG")
          encoded_image_bytes = img_byte_arr.getvalue()

          mask_data = lb_types.MaskData(im_bytes=encoded_image_bytes)
          mask_annotation = lb_types.ObjectAnnotation(
              name=ontology_mapping[class_name],
              value=lb_types.Mask(mask=mask_data, color=(255, 255, 255)),
          )
          annotations.append(mask_annotation)

    return annotations

##### Polygon

In [ ]:
def get_yolo_polygon_annotation_predictions(
        yolo_results, model,
        ontology_mapping: dict[str:str]) -> list[lb.Label]:
    """Convert YOLOv8 model results to labelbox polygon annotations format

    Args:
        yolo_result (Results): YOLOv8 prediction results.
        model (Model): YOLOv8 model.
        ontology_mapping (dict[<yolo_class_name>: <labelbox_feature_name>]): Allows mapping between YOLOv8 class names and different Labelbox feature names.
    Returns:
        list[lb_types.ObjectAnnotation]
    """
    annotations = []
    for yolo_result in yolo_results:
      for i, coordinates in enumerate(yolo_result.masks.xy):
          class_name = model.names[int(yolo_result.boxes[i].cls)]

          # ignore polygons that are not included in our mapping
          if not class_name in ontology_mapping.keys():
              continue

          polygon_annotation = lb_types.ObjectAnnotation(
              name=ontology_mapping[class_name],
              value=lb_types.Polygon(points=[
                  lb_types.Point(x=coordinate[0], y=coordinate[1])
                  for coordinate in coordinates
              ]),
          )
          annotations.append(polygon_annotation)

    return annotations

#### Creating our labels
Now that we have defined our functions to create our Labelbox annotations we can run each image through YOLOv8 to obtain our predictions and then use those results with our global keys to create our labels. 

In [ ]:
# label list that will be populated
labels = []

for i, global_key in enumerate(global_keys):
    annotations = []

    # make YOLOv8 predictions
    result = model.predict(url_list[i])

    # run result through each function and adding them to our annotation list
    annotations += get_yolo_bbox_annotation_predictions(result, model,
                                                        bbox_class_mapping)
    annotations += get_yolo_polygon_annotation_predictions(
        result, model, polygon_class_mapping)
    annotations += get_yolo_segment_annotation_predictions(
        result, model, mask_class_mapping)

    labels.append(
        lb_types.Label(data={"global_key": global_key},
                       annotations=annotations))

#### Import annotations to Labelbox
We have now created our labels and can import them towards our project. For more information on importing annotations visit our [import image annotations](https://docs.labelbox.com/reference/import-image-annotations) guide.

##### Option A: Upload to a labeling project as pre-labels (MAL)

In [ ]:
# upload MAL labels for this data row in project
upload_job = lb.MALPredictionImport.create_from_objects(
    client=client,
    project_id=project.uid,
    name="mal_job" + str(uuid.uuid4()),
    predictions=labels,
)
upload_job.wait_until_done()

print(f"Errors: {upload_job.errors}")
print(f"Status of uploads: {upload_job.statuses}")

##### Option B: Upload to a labeling project using ground truth

In [ ]:
# upload label for this data row in project
upload_job = lb.LabelImport.create_from_objects(
    client=client,
    project_id=project.uid,
    name="label_import_job" + str(uuid.uuid4()),
    labels=labels,
)
upload_job.wait_until_done

print("Errors:", upload_job.errors)
print("Status of uploads: ", upload_job.statuses)

## Clean up
Uncomment and run the cell below to optionally delete Labelbox objects created

In [ ]:
# batch.delete()
# project.delete()
# dataset.delete()